In [19]:
import json
from openai import OpenAI

In [20]:
with open('../data/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

In [21]:
documents[0]['question']

'Who found the answer to a search query collar george herbert essay?'

In [22]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [43]:
from elasticsearch import Elasticsearch

In [44]:

es_client = Elasticsearch('http://localhost:9200') 

In [45]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "context": {"type": "text"},
            "answer": {"type": "text"},
            "question": {"type": "text"}
        }
    }
}


In [46]:
index_name = "questions-related"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions-related'})

In [47]:
from tqdm.auto import tqdm

In [48]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 960/960 [00:24<00:00, 39.61it/s]


In [63]:
def elastic_search(query):
    # search_query = {
    #     "size": 5,
    #     "query": {
    #         "bool": {
    #             "must": {
    #                 "multi_match": {
    #                     "query": query,
    #                     "fields": ["question^3", "context^3", "answer"],
    #                     "type": "best_fields"
    #                 }
    #             }
    #         }
    #     }
    # }
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "context", "answer"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [64]:
query = 'How many percentage of the U.S. retail lingerie market did L-Brands account for in 2024?'

In [65]:
result_docs = elastic_search(query)

In [67]:

    
    print(result_docs[0]['answer'])

In 2016, L-Brands, the parent company of Victoria’s Secret, accounted for 83 percent of the U.S. retail lingerie market.


In [68]:
def elastic_search(query):
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "context^3", "answer"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    # search_query = {
    #     "size": 5,
    #     "query": {
    #         "bool": {
    #             "must": {
    #                 "multi_match": {
    #                     "query": query,
    #                     "fields": ["question^3", "context", "answer"],
    #                     "type": "best_fields"
    #                 }
    #             }
    #         }
    #     }
    # }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [69]:
result_docs = elastic_search(query)

In [71]:

    print(result_docs[0]['question'])
    print(result_docs[0]['answer'])
    print("-----------------------")

What percentage of the U.S. retail lingerie market did L-Brands, the parent company of Victoria’s Secret, account for in 2016?
In 2016, L-Brands, the parent company of Victoria’s Secret, accounted for 83 percent of the U.S. retail lingerie market.
-----------------------


In [72]:
def build_prompt(query, search_results):
    prompt_template = """
You're a chatbot assistant. Answer the QUESTION based on the CONTEXT from the database.
Use only the facts from the CONTEXT which including: context of the question, the question and answer when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"context: {doc['context']}\nquestion: {doc['question']}\nanswer: {doc['answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [73]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    print(prompt)
    answer = llm(prompt)
    return answer

In [74]:
query = 'How many percentage of the U.S. retail lingerie market did L-Brands account for in 2024?'
rag(query)

You're a chatbot assistant. Answer the QUESTION based on the CONTEXT from the database.
Use only the facts from the CONTEXT which including: context of the question, the question and answer when answering the QUESTION.

QUESTION: How many percentage of the U.S. retail lingerie market did L-Brands account for in 2024?

CONTEXT: 
context: Follow CBSMIAMI.COM: Facebook | Twitter
NEW YORK (CBSMiami) – While more traditional styles of women’s underwear dominate the market, a number of independent brands are catering to a much larger market.
At the New York headquarters of Lively, the unmentionables on display aren’t necessarily meant to be kept under wraps.
“These styles look like you could wear them outside, not just under your clothes,” said Karina Mitchell.
“And they’ve certainly been styled that way,” Alexandra Alquiza responded.
Underwear is big business, and Victoria’s Secret has helped shape the industry.
According to IBIS World, in 2016, L-Brands, the parent company of Victoria’s Se

'According to the context, L-Brands accounted for 83 percent of the U.S. retail lingerie market in 2016.'